# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 

from sklearn.metrics import classification_report, accuracy_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import completeness_score

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y[Y.columns[0]].unique()), random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = completeness_score(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [5]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , adjusted_rand_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , adjusted_rand_score

(31, 1.0)
(34, 1.0)
(35, 1.0)
(9, 0.6176108502340409)
(3, 0.517020343250687)
(1, 0.459501975281486)
(14, 0.4520491087370686)
(15, 0.4520319903543966)
(6, 0.44199368681334195)
(32, 0.44074102877494076)
(5, 0.4268292706776436)
(18, 0.3996731301297728)
(13, 0.34607282712179244)
(19, 0.3454149472412672)
(11, 0.34139611476194376)
(7, 0.32779535793544234)
(12, 0.32363802225179283)
(23, 0.29654402174266453)
(2, 0.2956606989132056)
(8, 0.2900064592357025)
(22, 0.28666370863171164)
(4, 0.28592982529182825)
(27, 0.2786293777442247)
(0, 0.2770288623707596)
(28, 0.26661799583719153)
(30, 0.2652556479942446)
(29, 0.25815060620049446)
(33, 0.24841727748592937)
(20, 0.24839062384568847)
(16, 0.2438654105909153)
(17, 0.24294780433778437)
(41, 0.23664570382468175)
(40, 0.23566649746954518)
(38, 0.23466668755121192)
(37, 0.2272430333150113)
(36, 0.22300021745402793)
(42, 0.21370607163147487)
(10, 0.20169174024259026)
(21, 0.19356083048511286)
(26, 0.19152709820688

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [8]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6813    0.1761    0.2798       619
      Backdoor     0.6125    0.0864    0.1515       567
           DoS     0.3182    0.2707    0.2925      4075
      Exploits     0.6361    0.8082    0.7119     11236
       Fuzzers     0.8965    0.8662    0.8810      6127
       Generic     0.9969    0.9862    0.9915     53886
Reconnaissance     0.9216    0.7462    0.8247      3385
     Shellcode     0.6648    0.6134    0.6381       388
         Worms     0.8000    0.2105    0.3333        38

      accuracy                         0.8910     80321
     macro avg     0.7253    0.5293    0.5672     80321
  weighted avg     0.8943    0.8910    0.8880     80321



In [9]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [10]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.9008201441088131 max accuracy
29 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [12]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('service', 0.3937261295910655)
('smeansz', 0.18210584592164342)
('sbytes', 0.17202572881089168)
('dmeansz', 0.08525039521801334)
('state', 0.06163311145111661)
('sttl', 0.03266957496547536)
('Dpkts', 0.03206856989992881)
('dttl', 0.030009623932826072)
('dwin', 0.0034710306996518408)
('ct_state_ttl', 0.0033249535821780266)
('swin', 0.003192069143924013)
('is_ftp_login', 0.00031958224559695304)
('ct_ftp_cmd', 0.0002033845376884518)
('is_sm_ips_ports', 0.0)


In [14]:
clf.score(x_train, y_train) 

0.9132266774567553

In [15]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.9008201441088131

In [16]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.5391    0.1394    0.2215       495
      Backdoor     0.8030    0.1130    0.1981       469
           DoS     0.5701    0.1119    0.1871      3235
      Exploits     0.6132    0.9234    0.7370      8944
       Fuzzers     0.9092    0.8411    0.8738      4846
       Generic     0.9974    0.9895    0.9934     43219
Reconnaissance     0.9219    0.7622    0.8345      2696
     Shellcode     0.6471    0.6854    0.6657       321
         Worms     0.7812    0.7812    0.7812        32

      accuracy                         0.9008     64257
     macro avg     0.7536    0.5941    0.6103     64257
  weighted avg     0.9058    0.9008    0.8880     64257

